In [51]:
import sys
import ast
import pandas as pd

In [52]:
listOfEvents = []
input_file = "out2.txt"
with open(input_file,"r") as f:
    for line in f:
        listOfEvents.append(ast.literal_eval(line))
df = pd.DataFrame.from_dict(listOfEvents)
df["ReadAddr"]=""
df["WriteAddr"]=""
df["AllocatedPages"]=""



In [53]:

df.dtypes

starttime           int64
address            object
size               object
type               object
cudatime           object
cudatimeoverlap    object
eventid             int64
ReadAddr           object
WriteAddr          object
AllocatedPages     object
dtype: object

In [54]:
# df.loc[df["kernelid"]==0,["ReadAddr"]]="anything"
# df[df["kernelid"]==0]["ReadAddr"]

In [55]:
nvbit_file = "../mem_trace_multilayer.txt"
with open(nvbit_file,"r") as f:
    for line in f:
        if "Read" in line:
            data = line.split(":")[1]
            data = data.split(",")
            kernelid = int(data[1])
            readAddr = [e.strip() for e in data[2:] if e != "0x0"]
            #print(readAddr)
            df.loc[(df["type"]!="allocate") & (df["eventid"]==kernelid),["ReadAddr"]]=str(readAddr)
        if "Write" in line:
            data = line.split(":")[1]
            data = data.split(",")
            kernelid = int(data[1])
            writeAddr = [e.strip() for e in data[2:] if e != "0x0"]
            #print(writeAddr)
            df.loc[(df["type"]!="allocate") & (df["eventid"]==kernelid),["WriteAddr"]]=str(writeAddr)
            


In [56]:
listOfEvents = []
input_file_nvbit = "out2_nvbit.txt"
with open(input_file_nvbit,"r") as f:
    for line in f:
        listOfEvents.append(ast.literal_eval(line))
df_realaddr = pd.DataFrame.from_dict(listOfEvents)

In [57]:
realaddr = df_realaddr["address"]
for i, ra in enumerate(realaddr):
    df.loc[(df["type"]=="allocate") & (df["eventid"] == i),["address"]]=ra

In [58]:
def get_pagesAllocated(addr, size):
    if size < 0:
        return ""
    else:
        pages_to_return = []
        maxAddr = addr + size
        pageAddr = addr // 4096 *4096
        while pageAddr <= maxAddr:
            pages_to_return.append(hex(pageAddr))
            pageAddr += 4096
    return pages_to_return

In [59]:
df_realaddr['address'] = df_realaddr['address'].apply(lambda x:int(x,0))
df_realaddr['size'] = df_realaddr['size'].astype('int')
df_realaddr[['address','size']]


,address,size
0,1476130688,8
1,1476227904,8
2,1460523904,400000
3,1476230848,8
4,1476230848,-8
...,...,...
77,140133984632832,2048
78,140133961826304,1644032
79,140133961826304,-1644032
80,140136456139264,512


In [60]:
df_realaddr['col_3'] = df_realaddr.apply(lambda x: str(get_pagesAllocated(x['address'], x['size'])), axis=1)

In [61]:
df_realaddr['col_3']

0                                        ['0x57fbf000']
1                                        ['0x57fd7000']
2     ['0x570dd000', '0x570de000', '0x570df000', '0x...
3                                        ['0x57fd7000']
4                                                      
                            ...                        
77                                   ['0x7f737c600000']
78    ['0x7f737b040000', '0x7f737b041000', '0x7f737b...
79                                                     
80                                   ['0x7f740fb03000']
81                                                     
Name: col_3, Length: 82, dtype: object

In [62]:
pageaddr = df_realaddr["col_3"]
for i, pa in enumerate(pageaddr):
    df.loc[(df["type"]=="allocate") & (df["eventid"] == i),["AllocatedPages"]]=pa

In [63]:
df.to_csv(input_file.split('.')[0]+"2.csv")